# Simple line example

In this example we will examine the different ways of fitting a line using the easyCore framework.

A line is defined by the parameters `m` and `c`, the gradient and intercept.

In [ ]:
from easyCore import np
from easyCore.Objects.ObjectClasses import Parameter, BaseObj
from easyCore.Fitting.Fitting import Fitter
import holoviews as hv
hv.extension('bokeh')

## Using the `BaseObj` class directly

The first way of using EasyCore, is to directly useing the `BaseObj` class.

The `BaseObj` class is what most objects are based  off of. It takes either a child of the `Descriptor` or `BaseObj` class and integrates it as a variable in itself. For the example of a Line, we supply the `Parameters` _m_ and _c_.

In [ ]:
Line = BaseObj('Line', m=Parameter('m', 1), c=Parameter('c', 1))

Since this is not a class, the calculation appears outside the definition.

In [ ]:
def line_func(line_obj, x):
    return line_obj.m.raw_value * x + line_obj.c.raw_value

Lets create some random data with a normal distribution...

In [ ]:
x = np.linspace(0, 10, 101)
real_m = 4.5
real_c= 2.8
y = real_m * x + real_c + 2*(1 - np.random.rand(x.size))

In [ ]:
hv.Curve((x, y), (x, line_func(Line, x)))

This can now be fitted using lmfits least squares algorith

In [ ]:
# Initialize the EasyCore fitter
f = Fitter()
# Set the object and function
f.initialize(Line, line_func)
f_res = f.fit(x, y)
print(f_res.goodness_of_fit)

And display the results

In [ ]:
hv.Curve((x, y), (x, line_func(Line, x)))

## SubClassing `BaseObj`

This is the preferred way of working with `Core` objects as you can keep everything contained.

Lets redefine a line...

In [ ]:
from typing import Optional, Union # We like typing, it helps everyone code :-)


class Line(BaseObj):
    def __init__(self, m: Optional[Union[Parameter, float]] = None, c: Optional[Union[Parameter, float]] = None):
        super(Line, self).__init__('Line', m=Parameter('m', 1.0), c=Parameter('c', 1.0))
        if m:
            self.m = m
        if c:
            self.c = c
    def __call__(self, x, *args, **kwargs):
        return self.m.raw_value * x + self.c.raw_value

We have now created an EasyCore class called `Line` which has inherited all the useful functions of `BaseObj`. For example...

In [ ]:
l1 = Line(2, 3)
print(l1)

In [ ]:
l1.get_fit_parameters()

And access all components

In [ ]:
print(l1.m)
print(l1.c)

Now fitting is just:

In [ ]:
f = Fitter(l1, l1.__call__)
f_res = f.fit(x, y)
print(f_res.goodness_of_fit)
hv.Curve((x, y), (x, l1(x)))

In [ ]:
print(l1)

## using a different minimizer

By default, `EasyCore` has all minimizers in the `lmfit` package. But interfaces to the `bumps` and `DFO_LS` and their associated minimizers also exist. It is relatively simple to use them. In this example we will use the Dream minimizer in the `bumps` package.

In [ ]:
l2 = Line()
f = Fitter(l2, l2.__call__)
f.switch_engine('bumps')
method = 'dream'
dream_kwargs = {
    k: v for k, v in [('samples', 10000), ('burn', 100), ('pop', 10), ('init', 'eps'), ('thin', 1), ('alpha', 0.01), ('outliers', 'none'), ('trim', False), ('steps', 0)]
}

f_res = f.fit(x, y, method='dream', minimizer_kwargs=dream_kwargs)

The results of which are:

In [ ]:
print(f_res.goodness_of_fit)
hv.Curve((x, y), (x, l2(x)))